In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# --- 설정 ---
SAMPLE_SIZE = 6000
INPUT_FILENAME = "normal_dataset(콜센터 상담).csv"
OUTPUT_FILENAME_PREFIX = "stratified_sample_callcenter"
# --- 설정 끝 ---

# 파일 경로 설정
input_file = os.path.join("dataset", INPUT_FILENAME)
output_file = os.path.join("dataset", f"{OUTPUT_FILENAME_PREFIX}_{SAMPLE_SIZE}.csv")

# 데이터 불러오기
print(f"'{input_file}'에서 데이터를 불러오는 중...")
try:
    df = pd.read_csv(input_file)
    print("데이터 불러오기 완료.")
except FileNotFoundError:
    print(f"오류: '{input_file}'을(를) 찾을 수 없습니다. 경로를 확인하세요.")
    exit()

# 고유 file_name과 해당 category/subcategory 추출
print("고유 file_name을 기준으로 데이터 집계 중...")
unique_files = (
    df[["file_name", "category", "subcategory"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
print(f"총 {len(unique_files)}개의 고유한 file_name을 찾았습니다.")

# 데이터가 요청된 샘플 수보다 적은 경우, 가능한 모든 데이터를 사용
if len(unique_files) < SAMPLE_SIZE:
    print(
        f"경고: 고유 file_name의 수가 {SAMPLE_SIZE}개({len(unique_files)}개)보다 적습니다."
    )
    print(
        f"가능한 모든 {len(unique_files)}개의 고유 file_name을 사용하여 파일을 생성합니다."
    )
    actual_sample_size = len(unique_files)
    output_file = os.path.join(
        "dataset", f"{OUTPUT_FILENAME_PREFIX}_{actual_sample_size}.csv"
    )
    sampled_files_df = unique_files
else:
    actual_sample_size = SAMPLE_SIZE
    # category와 subcategory를 합쳐서 계층화 키로 사용
    unique_files["stratify_key"] = (
        unique_files["category"].astype(str)
        + "_"
        + unique_files["subcategory"].astype(str)
    )

    # 계층적 샘플링 수행
    print(f"{actual_sample_size}개에 대한 계층적 샘플링 수행 중...")
    try:
        sampled_files_df, _ = train_test_split(
            unique_files,
            train_size=actual_sample_size,
            stratify=unique_files["stratify_key"],
            random_state=42,
        )
        print("샘플링 완료.")
    except ValueError as e:
        print(f"샘플링 오류: {e}")
        print(
            "계층별 최소 샘플 수가 부족할 수 있습니다. stratify 없이 샘플링을 시도합니다."
        )
        sampled_files_df = unique_files.sample(n=actual_sample_size, random_state=42)

# 샘플링된 file_name 리스트 추출
sampled_file_names = sampled_files_df["file_name"].tolist()

# 원본 데이터프레임에서 샘플링된 file_name에 해당하는 모든 데이터 추출
print("원본 데이터에서 샘플링된 데이터 추출 중...")
result_df = df[df["file_name"].isin(sampled_file_names)]

# 결과를 CSV 파일로 저장
print(f"결과를 '{output_file}'에 저장 중...")
result_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print("\n처리 완료!")
print(f"총 {len(result_df)}개의 행이 포함된 샘플 데이터가 생성되었습니다.")
print(f"고유 file_name 개수: {len(sampled_file_names)}개")
print(f"결과 파일: '{os.path.abspath(output_file)}'")

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# 파일 경로 설정
# 스크립트가 프로젝트 루트에서 실행된다고 가정합니다.
input_file = os.path.join("dataset", "normal_dataset3.csv")
output_file = os.path.join("dataset", "stratified_8000_sample.csv")

# 데이터 불러오기
print(f"'{input_file}'에서 데이터를 불러오는 중...")
try:
    df = pd.read_csv(input_file)
    print("데이터 불러오기 완료.")
except FileNotFoundError:
    print(f"오류: '{input_file}'을(를) 찾을 수 없습니다. 경로를 확인하세요.")
    exit()

# 고유 file_name과 해당 category/subcategory 추출
# file_name 당 category와 subcategory는 동일하다고 가정
print("고유 file_name을 기준으로 데이터 집계 중...")
unique_files = (
    df[["file_name", "category", "subcategory"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
print(f"총 {len(unique_files)}개의 고유한 file_name을 찾았습니다.")

SAMPLE_SIZE = 8000

# 데이터가 요청된 샘플 수보다 적은 경우, 가능한 모든 데이터를 사용
if len(unique_files) < SAMPLE_SIZE:
    print(
        f"경고: 고유 file_name의 수가 {SAMPLE_SIZE}개({len(unique_files)}개)보다 적습니다."
    )
    print(
        f"가능한 모든 {len(unique_files)}개의 고유 file_name을 사용하여 파일을 생성합니다."
    )
    SAMPLE_SIZE = len(unique_files)
    output_file = os.path.join("dataset", f"stratified_{SAMPLE_SIZE}_sample.csv")
    sampled_files_df = unique_files
else:
    # category와 subcategory를 합쳐서 계층화 키로 사용
    unique_files["stratify_key"] = (
        unique_files["category"].astype(str)
        + "_"
        + unique_files["subcategory"].astype(str)
    )

    # 계층적 샘플링 수행
    print(f"{SAMPLE_SIZE}개에 대한 계층적 샘플링 수행 중...")
    try:
        sampled_files_df, _ = train_test_split(
            unique_files,
            train_size=SAMPLE_SIZE,
            stratify=unique_files["stratify_key"],
            random_state=42,
        )
        print("샘플링 완료.")
    except ValueError as e:
        print(f"샘플링 오류: {e}")
        print(
            "계층별 최소 샘플 수가 부족할 수 있습니다. stratify 없이 샘플링을 시도합니다."
        )
        sampled_files_df = unique_files.sample(n=SAMPLE_SIZE, random_state=42)


# 샘플링된 file_name 리스트 추출
sampled_file_names = sampled_files_df["file_name"].tolist()

# 원본 데이터프레임에서 샘플링된 file_name에 해당하는 모든 데이터 추출
print("원본 데이터에서 샘플링된 데이터 추출 중...")
result_df = df[df["file_name"].isin(sampled_file_names)]

# 결과를 CSV 파일로 저장
print(f"결과를 '{output_file}'에 저장 중...")
result_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print("\n처리 완료!")
print(f"총 {len(result_df)}개의 행이 포함된 샘플 데이터가 생성되었습니다.")
print(f"고유 file_name 개수: {len(sampled_file_names)}개")
print(f"결과 파일: '{os.path.abspath(output_file)}'")

# 샘플링된 데이터의 category 및 subcategory 분포 확인
print("\n샘플링된 데이터의 Category 분포:")
print(result_df["category"].value_counts(normalize=True))
print("\n샘플링된 데이터의 Subcategory 분포:")
print(result_df["subcategory"].value_counts(normalize=True))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

# --- 설정 --- #
SAMPLE_SIZE = 2000
INPUT_FILENAME = "normal_dataset2(감정분류).csv"
OUTPUT_FILENAME_PREFIX = "stratified_sample_emotion"
# --- 설정 끝 --- #

# 파일 경로 설정
input_file = os.path.join("dataset", INPUT_FILENAME)
output_file = os.path.join("dataset", f"{OUTPUT_FILENAME_PREFIX}_{SAMPLE_SIZE}.csv")

# 데이터 불러오기
print(f"'{input_file}'에서 데이터를 불러오는 중...")
try:
    df = pd.read_csv(input_file)
    print("데이터 불러오기 완료.")
except FileNotFoundError:
    print(f"오류: '{input_file}'을(를) 찾을 수 없습니다. 경로를 확인하세요.")
    exit()

# 고유 file_name과 해당 category/subcategory 추출
print("고유 file_name을 기준으로 데이터 집계 중...")
unique_files = (
    df[["file_name", "category", "subcategory"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
print(f"총 {len(unique_files)}개의 고유한 file_name을 찾았습니다.")

# 데이터가 요청된 샘플 수보다 적은 경우, 가능한 모든 데이터를 사용
if len(unique_files) < SAMPLE_SIZE:
    print(
        f"경고: 고유 file_name의 수가 {SAMPLE_SIZE}개({len(unique_files)}개)보다 적습니다."
    )
    print(
        f"가능한 모든 {len(unique_files)}개의 고유 file_name을 사용하여 파일을 생성합니다."
    )
    actual_sample_size = len(unique_files)
    output_file = os.path.join(
        "dataset", f"{OUTPUT_FILENAME_PREFIX}_{actual_sample_size}.csv"
    )
    sampled_files_df = unique_files
else:
    actual_sample_size = SAMPLE_SIZE
    # category와 subcategory를 합쳐서 계층화 키로 사용
    unique_files["stratify_key"] = (
        unique_files["category"].astype(str)
        + "_"
        + unique_files["subcategory"].astype(str)
    )

    # 계층적 샘플링 수행
    print(f"{actual_sample_size}개에 대한 계층적 샘플링 수행 중...")
    try:
        sampled_files_df, _ = train_test_split(
            unique_files,
            train_size=actual_sample_size,
            stratify=unique_files["stratify_key"],
            random_state=42,
        )
        print("샘플링 완료.")
    except ValueError as e:
        print(f"샘플링 오류: {e}")
        print(
            "계층별 최소 샘플 수가 부족할 수 있습니다. stratify 없이 샘플링을 시도합니다."
        )
        sampled_files_df = unique_files.sample(n=actual_sample_size, random_state=42)

# 샘플링된 file_name 리스트 추출
sampled_file_names = sampled_files_df["file_name"].tolist()

# 원본 데이터프레임에서 샘플링된 file_name에 해당하는 모든 데이터 추출
print("원본 데이터에서 샘플링된 데이터 추출 중...")
result_df = df[df["file_name"].isin(sampled_file_names)]

# 결과를 CSV 파일로 저장
print(f"결과를 '{output_file}'에 저장 중...")
result_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print("\n처리 완료!")
print(f"총 {len(result_df)}개의 행이 포함된 샘플 데이터가 생성되었습니다.")
print(f"고유 file_name 개수: {len(sampled_file_names)}개")
print(f"결과 파일: '{os.path.abspath(output_file)}'")

# file_name 간략화

In [ ]:
import pandas as pd
import os

# --- 설정 --- #
# 각 파일에 대한 처리 규칙을 정의합니다.
# 형식: (파일명, 새 이름 접두사)
TASKS = [
    ("stratified_2862_sample.csv", "일반통화"),
    ("stratified_sample_callcenter_6000.csv", "콜센터"),
    ("stratified_sample_emotion_2000.csv", "상담대화"),
]

DATASET_DIR = "dataset"
FILENAME_COLUMN = "file_name"
# --- 설정 끝 --- #


def rename_file_names_in_csv(file_path, prefix, column_name):
    """CSV 파일 내의 지정된 컬럼 값을 새로운 접두사와 숫자로 변경합니다."""
    print(f"'{file_path}' 파일 처리 중...")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"-> 파일을 찾을 수 없습니다. 건너뜁니다.")
        return

    if column_name not in df.columns:
        print(f"-> '{column_name}' 컬럼을 찾을 수 없습니다. 건너뜁니다.")
        return

    # 고유한 기존 file_name 목록 가져오기
    unique_names = df[column_name].unique()

    # 기존 file_name을 새 이름으로 매핑하는 딕셔너리 생성
    # 예: {'original_name_A': '콜센터1', 'original_name_B': '콜센터2', ...}
    name_map = {old_name: f"{prefix}{i+1}" for i, old_name in enumerate(unique_names)}

    # 매핑을 사용하여 컬럼 값 변경
    df[column_name] = df[column_name].map(name_map)

    # 변경된 데이터프레임을 동일한 파일에 덮어쓰기
    df.to_csv(file_path, index=False, encoding="utf-8-sig")
    print(
        f"-> 완료. {len(unique_names)}개의 고유한 file_name이 새로운 형식으로 변경되었습니다."
    )


if __name__ == "__main__":
    print("file_name 변경 작업을 시작합니다.")
    # 프로젝트 루트에 있는 dataset 폴더를 기준으로 작업
    base_path = os.path.join(".", DATASET_DIR)

    for filename, prefix in TASKS:
        file_path = os.path.join(base_path, filename)
        rename_file_names_in_csv(file_path, prefix, FILENAME_COLUMN)

    print("\n모든 작업이 완료되었습니다.")

# 3개의 데이터셋 통합

In [1]:
import pandas as pd
import os
import glob

# --- 설정 --- #
# CSV 파일이 있는 디렉토리
SOURCE_DIR = "dataset"
# 병합된 파일을 저장할 이름
OUTPUT_FILENAME = "merged_dataset.csv"
# --- 설정 끝 --- #


def merge_all_csv_in_directory(source_dir, output_filename):
    """지정된 디렉토리의 모든 CSV 파일을 하나로 병합합니다."""
    # 프로젝트 루트를 기준으로 소스 디렉토리 경로 설정
    search_path = os.path.join(".", source_dir, "*.csv")
    output_path = os.path.join(".", source_dir, output_filename)

    # 소스 디렉토리에서 모든 CSV 파일 목록 가져오기
    all_csv_files = glob.glob(search_path)

    # 만약 스크립트가 이전에 실행되어 병합된 파일이 존재한다면, 목록에서 제외
    if output_path in all_csv_files:
        print(f"기존에 병합된 파일 '{output_path}'을(를) 제외합니다.")
        all_csv_files.remove(output_path)

    if not all_csv_files:
        print("병합할 CSV 파일이 없습니다.")
        return

    print("다음 파일들을 병합합니다:")
    for f in all_csv_files:
        print(f" - {f}")

    # 모든 CSV 파일을 읽어 데이터프레임 리스트에 추가
    dataframe_list = []
    for f in all_csv_files:
        try:
            df = pd.read_csv(f)
            dataframe_list.append(df)
        except Exception as e:
            print(f"'{f}' 파일을 읽는 중 오류 발생: {e}")

    if not dataframe_list:
        print("데이터를 성공적으로 읽은 파일이 없습니다.")
        return

    # 모든 데이터프레임 병합
    print("\n데이터프레임 병합 중...")
    merged_df = pd.concat(dataframe_list, ignore_index=True)

    # 결과를 새 CSV 파일로 저장
    print(f"병합된 데이터를 '{output_path}'에 저장 중...")
    merged_df.to_csv(output_path, index=False, encoding="utf-8-sig")

    print("\n--- 작업 완료 ---")
    print(f"총 {len(all_csv_files)}개의 파일이 병합되었습니다.")
    print(
        f"병합된 파일 '{output_path}'에는 총 {len(merged_df)}개의 행이 포함되어 있습니다."
    )


if __name__ == "__main__":
    merge_all_csv_in_directory(SOURCE_DIR, OUTPUT_FILENAME)

병합할 CSV 파일이 없습니다.


# 데이터셋 총 데이터 갯수

In [ ]:
import pandas as pd
import os

# 파일 경로 설정
file_path = os.path.join("dataset", "merged_dataset.csv")

# 데이터 불러오기
print(f"'{file_path}'에서 데이터를 불러오는 중...")
try:
    df = pd.read_csv(file_path)
    # 고유한 file_name 개수 세기
    unique_count = df["file_name"].nunique()
    print("--- 결과 ---")
    print(f"고유한 file_name의 개수는 {unique_count}개입니다.")
except FileNotFoundError:
    print(f"오류: '{file_path}'을(를) 찾을 수 없습니다.")
except Exception as e:
    print(f"오류 발생: {e}")

# 데이터셋에 있는 데이터 통계

In [ ]:
import pandas as pd
import os

# --- 설정 --- #
INPUT_FILE = "merged_dataset.csv"
# --- 설정 끝 --- #

# 파일 경로 설정
file_path = os.path.join("dataset", INPUT_FILE)

# 데이터 불러오기
print(f"'{file_path}'에서 데이터를 불러오는 중...")
try:
    df = pd.read_csv(file_path)
    print("데이터 불러오기 완료.")
except FileNotFoundError:
    print(f"오류: '{file_path}'을(를) 찾을 수 없습니다.")
    exit()
except Exception as e:
    print(f"데이터를 읽는 중 오류 발생: {e}")
    exit()

# 동일한 file_name의 msg를 하나의 대화로 묶기
print("동일한 file_name의 msg를 하나의 대화로 묶는 중...")
# 각 file_name별로 msg를 공백으로 연결
conversations = (
    df.groupby("file_name")["msg"]
    .apply(lambda x: " ".join(x.astype(str)))
    .reset_index()
)
conversations.rename(columns={"msg": "full_conversation"}, inplace=True)

# 대화의 글자 길이 계산
print("대화의 글자 길이 계산 중...")
conversations["conversation_length"] = conversations["full_conversation"].apply(len)

# 글자 길이 통계 확인
print("\n--- 대화 글자 길이 통계 --- ")
print(conversations["conversation_length"].describe())

print("\n분석 완료.")